In [1]:
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from openai import OpenAI

# Setting up my Hugging Face token and model ID
HF_TOKEN = "Use you API Key"
MODEL_ID = "deepseek-ai/DeepSeek-R1-0528:novita"

# Initializing the OpenAI-compatible Hugging Face client
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)

# Reading my ACT theory article so I can use it for retrieval
with open("article.txt", "r", encoding="utf-8", errors="ignore") as f:
    full_text = f.read()

# Splitting the article into smaller chunks for easier processing
def split_text(text, max_tokens=500):
    paragraphs = text.split("\n")
    chunks, current = [], ""
    token_estimate = lambda x: len(x.split())
    for p in paragraphs:
        p = p.strip()
        if not p:
            continue
        if token_estimate(current + p) < max_tokens:
            current += " " + p
        else:
            chunks.append(current.strip())
            current = p
    if current:
        chunks.append(current.strip())
    return chunks

chunks = split_text(full_text)
print(f"Split into {len(chunks)} chunks.")

# Creating embeddings for each chunk and building a FAISS index
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.encode(chunks, convert_to_numpy=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype("float32"))

# Saving the FAISS index and chunk metadata for later use
faiss.write_index(index, "act_article_index.faiss")
with open("act_chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, indent=2)

print("Saved index and chunks.")

# Reading the therapy session transcript and truncating to 500 words
with open("Session One.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_text = f.read()

session_text = ' '.join(session_text.split()[:500])

# Creating a query to guide the summarization using ACT principles
query = (
    "Summarize the following therapy session using ACT principles. "
    "Focus on how the therapist and client address values, avoidance, acceptance, and defusion:\n\n"
    + session_text
)

# Finding the most relevant ACT chunks for the query
query_embedding = embedding_model.encode([query], convert_to_numpy=True)
k = 3
D, I = index.search(np.array(query_embedding).astype("float32"), k)
retrieved_chunks = [chunks[i] for i in I[0]]
context = "\n\n".join(retrieved_chunks)

# Building system and user prompts with the retrieved context
system_prompt = "You are a clinical psychologist summarizing therapy sessions using ACT (Acceptance and Commitment Therapy) principles."

user_prompt = f"""
Using the following ACT theory context:
---
{context}
---

Summarize the therapy session below by identifying and explaining ACT principles reflected:
---
{session_text}
---
"""

# Sending the summarization request to the Hugging Face router
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0.4,
    max_tokens=800,
)

# Printing the final ACT-informed summary
summary = response.choices[0].message.content.strip()
print("\nACT-Informed Summary:\n")
print(summary)


Split into 12 chunks.


C:\Users\Gautham Rajesh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Saved index and chunks.

ACT-Informed Summary:

<think>
Hmm, the user wants me to summarize a therapy session using ACT principles. They've provided extensive background on ACT theory and a transcript of a session between a therapist (T) and client (C).

Looking at the session, I notice several key ACT principles being demonstrated. The client expresses worry about not fully understanding the therapy approach, which shows cognitive fusion (taking thoughts literally) and experiential avoidance (discomfort with uncertainty). The therapist skillfully addresses this by distinguishing between descriptive language and functional language - this relates to ACT's cognitive defusion principle.

The therapist uses the sports analogy (ski instructor talking too much) to illustrate how overthinking creates obstacles. This demonstrates contact with the present moment - encouraging direct experience over excessive analysis. The therapist explicitly values flexibility ("freedom to be inconsistent") o

In [26]:
from openai import OpenAI
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# Setting up my Hugging Face token and model ID
HF_TOKEN = "Use you API Key"
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct:cerebras"

# Initializing the OpenAI-compatible Hugging Face client
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)

# Reading the ACT theory article so I can use it for retrieval
with open("article.txt", "r", encoding="utf-8", errors="ignore") as f:
    full_text = f.read()

# Splitting the article into smaller chunks for easier processing
def split_text(text, max_tokens=500):
    paragraphs = text.split("\n")
    chunks, current = [], ""
    token_estimate = lambda x: len(x.split())
    for p in paragraphs:
        p = p.strip()
        if not p:
            continue
        if token_estimate(current + p) < max_tokens:
            current += " " + p
        else:
            chunks.append(current.strip())
            current = p
    if current:
        chunks.append(current.strip())
    return chunks

chunks = split_text(full_text)
print(f"Split into {len(chunks)} chunks.")

# Creating embeddings for each chunk and building a FAISS index
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.encode(chunks, convert_to_numpy=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype("float32"))

# Saving the FAISS index and chunk metadata for later use
faiss.write_index(index, "act_article_index.faiss")
with open("act_chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, indent=2)

print("Saved index and chunks.")

# Reading the therapy session transcript and truncating to 500 words
with open("Session One.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_text = f.read()

session_text = ' '.join(session_text.split()[:500])

# Creating a query to guide the summarization using ACT principles
query = (
    "Summarize the following therapy session using ACT principles. "
    "Focus on how the therapist and client address values, avoidance, acceptance, and defusion:\n\n"
    + session_text
)

# Finding the most relevant ACT chunks for the query
query_embedding = embedding_model.encode([query], convert_to_numpy=True)
k = 3
D, I = index.search(np.array(query_embedding).astype("float32"), k)
retrieved_chunks = [chunks[i] for i in I[0]]
context = "\n\n".join(retrieved_chunks)

# Building system and user prompts with the retrieved context
system_prompt = "You are a clinical psychologist summarizing therapy sessions using ACT (Acceptance and Commitment Therapy) principles."

user_prompt = f"""
Using the following ACT theory context:
---
{context}
---

Summarize the therapy session below by identifying and explaining ACT principles reflected:
---
{session_text}
---
"""

# Sending the summarization request to the Hugging Face model
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0.4,
    max_tokens=800,
)

# Printing the final ACT-informed summary
summary = response.choices[0].message.content.strip()
print("\nACT-Informed Summary:\n")
print(summary)


Split into 12 chunks.
Saved index and chunks.

ACT-Informed Summary:

In this therapy session, several ACT principles are reflected:

1. **Cognitive Defusion**: The therapist (T) helps the client (C) to see that language can be used in different ways, such as describing something (reporter) or making a difference (coach). This helps to reduce the tendency to reify thoughts and focus on the literal meaning of words.

Example: "One kind of talk is about describing something. The other kind of talk is to make a difference. And sometimes the one isn’t the best avenue to the other."

2. **Acceptance**: The therapist encourages the client to be open to new ideas and perspectives, even if they are unfamiliar or contradictory. This helps the client to accept their experiences and thoughts without judgment.

Example: "I want us to talk to make a difference. Do you play any kind of sports or music?"

3. **Contact with the Present Moment**: The therapist helps the client to focus on the present m

In [28]:
from openai import OpenAI
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# Setting up my Hugging Face token and the model ID
HF_TOKEN = "Use you API Key"
MODEL_ID = "moonshotai/Kimi-K2-Instruct:novita"

# Initializing the Hugging Face client with OpenAI API compatibility
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)

# Reading the ACT theory article for use in retrieval
with open("article.txt", "r", encoding="utf-8", errors="ignore") as f:
    full_text = f.read()

# Splitting the article into smaller chunks for indexing
def split_text(text, max_tokens=500):
    paragraphs = text.split("\n")
    chunks, current = [], ""
    token_estimate = lambda x: len(x.split())
    for p in paragraphs:
        p = p.strip()
        if not p:
            continue
        if token_estimate(current + p) < max_tokens:
            current += " " + p
        else:
            chunks.append(current.strip())
            current = p
    if current:
        chunks.append(current.strip())
    return chunks

chunks = split_text(full_text)
print(f"Split into {len(chunks)} chunks.")

# Creating embeddings for each chunk and building a FAISS index
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.encode(chunks, convert_to_numpy=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype("float32"))

# Saving the FAISS index and chunk metadata
faiss.write_index(index, "act_article_index.faiss")
with open("act_chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, indent=2)

print("Saved index and chunks.")

# Reading the therapy session transcript and truncating it to 500 words
with open("Session One.txt", "r", encoding="utf-8", errors="ignore") as f:
    session_text = f.read()

session_text = ' '.join(session_text.split()[:500])

# Creating a query to guide summarization using ACT principles
query = (
    "Summarize the following therapy session using ACT principles. "
    "Focus on how the therapist and client address values, avoidance, acceptance, and defusion:\n\n"
    + session_text
)

# Finding the most relevant ACT chunks for the query
query_embedding = embedding_model.encode([query], convert_to_numpy=True)
k = 3
D, I = index.search(np.array(query_embedding).astype("float32"), k)
retrieved_chunks = [chunks[i] for i in I[0]]
context = "\n\n".join(retrieved_chunks)

# Building system and user prompts with the retrieved context
system_prompt = "You are a clinical psychologist summarizing therapy sessions using ACT (Acceptance and Commitment Therapy) principles."

user_prompt = f"""
Using the following ACT theory context:
---
{context}
---

Summarize the therapy session below by identifying and explaining ACT principles reflected:
---
{session_text}
---
"""

# Sending the summarization request to the Hugging Face model
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0.4,
    max_tokens=800,
)

# Printing the final ACT-informed summary
summary = response.choices[0].message.content.strip()
print("\nACT-Informed Summary:\n")
print(summary)


Split into 12 chunks.
Saved index and chunks.

ACT-Informed Summary:

ACT Principles Demonstrated in Session One

1. Cognitive Defusion  
   • The therapist reframes the client’s demand for a detailed, logical description of ACT as a form of cognitive fusion—treating words as literal truths rather than tools.  
   • Metaphors (reporter vs. coach, skiing instructions) are used to loosen the grip of literal language so the client can notice thoughts without being ruled by them.

2. Acceptance & Willingness  
   • The client openly reports worry and dislike of uncertainty. The therapist does not try to reduce or argue against these feelings; instead, the exchange models acceptance of discomfort in the service of pursuing therapy itself.

3. Present-Moment Focus & Experiential Work  
   • Rather than explaining ACT conceptually, the therapist keeps attention on the immediate, felt experience of “talk that makes a difference” versus “talk that describes.”  
   • The skiing example grounds t